In [1]:
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:

def loadData():

    data = pd.read_csv("E202-COMP7117-TD01-00 - classification.csv")

    if data.isna().values.any() == True:
        data = data.dropna()
    
    dataInput = data[["volatile acidity", "chlorides", "free sulfur dioxide", "total sulfur dioxide", 
                      "density", "pH","sulphates","alcohol"]]
    target = data[["quality"]]
    
    print(dataInput.shape)
    
    for y in dataInput:
        if(y == "free sulfur dioxide" ):
#             print(y)
#             print(dataInput[y][0])
            for j in range(len(dataInput)):
                if(dataInput[y][j] == "High"):
                    dataInput.at[j, y] = 3
                elif(dataInput[y][j] == "Medium"):
                    dataInput.at[j, y] = 2
                elif(dataInput[y][j] == "Low"):
                    dataInput.at[j, y] = 1
                else:
                    dataInput.at[j, y] = 0
        elif(y == "density"):
#             print(y)
#             print(dataInput[y][0])
            for j in range(len(dataInput)):
                if(dataInput[y][j] == "Very High"):
                    dataInput.at[j, y] = 0
                elif(dataInput[y][j] == "High"):
                    dataInput.at[j, y] = 3
                elif(dataInput[y][j] == "Medium"):
                    dataInput.at[j, y] = 2
                elif(dataInput[y][j] == "Low"):
                    dataInput.at[j, y] = 1
        elif(y == "pH"):
#             print(y)
#             print(dataInput[[y]].values[0])
            for j in range(len(dataInput)):
                    if(dataInput[y][j] == "Very Basic"):
                        dataInput.at[j, y] = 3
                    elif(dataInput[y][j] == "Normal"):
                        dataInput.at[j, y] = 2
                    elif(dataInput[y][j] == "Very Acidic"):
                        dataInput.at[j, y] = 1
                    else:
                        dataInput.at[j, y] = 0
    
    #normalisasi
    feature = MinMaxScaler().fit_transform(dataInput)
    target = OneHotEncoder(sparse=False).fit_transform(target)
    
    #PCA
    result = PCA(n_components=4).fit_transform(dataInput)
        
    return result, target


inputData, target = loadData()
print(inputData)

(1599, 8)
[[-1.45528465e+07 -3.05014950e+00 -1.24457169e+01  1.69498961e+00]
 [-1.45528461e+07 -3.23163925e+00  2.05665151e+01  6.30518659e-01]
 [-1.45528461e+07 -3.20840843e+00  7.54834329e+00  2.09272675e+00]
 ...
 [-1.45528449e+07 -3.30578440e+00 -6.43626411e+00 -2.88568481e-01]
 [-1.45528457e+07 -3.21475605e+00 -2.43687977e+00 -9.87204246e-01]
 [-1.45528449e+07 -3.52652057e+00 -4.45736411e+00  3.54442272e-01]]


In [8]:

layers = {
    "input": 4, #8 different kind of data
    "hidden": 500,
    "output": 5 # decent, fair, fine, good, great
}

weights = {
    'input_to_hidden' : tf.Variable(tf.random_normal([layers['input'], layers['hidden']])),
    'hidden_to_output' : tf.Variable(tf.random_normal([layers['hidden'], layers['output']]))
}

bias = {
    'input_to_hidden' : tf.Variable(tf.random_normal([layers['hidden']])),
    'hidden_to_output' : tf.Variable(tf.random_normal([layers['output']]))
}

inputPlaceholder = tf.placeholder(tf.float32, [None, layers["input"]])
outputPlacehlder = tf.placeholder(tf.float32, [None, layers["output"]])


In [28]:

def feedForward(inputData):
    #first
    input_to_hidden_bias = tf.matmul(inputData, weights['input_to_hidden']) + bias['input_to_hidden']
    activated_input_to_hidden = tf.nn.sigmoid(input_to_hidden_bias)
    #second
    hidden_to_output_bias = tf.matmul(activated_input_to_hidden, weights['hidden_to_output']) + bias['hidden_to_output']
    activated_hidden_to_output = tf.nn.sigmoid(hidden_to_output_bias)

    return activated_hidden_to_output

predict = feedForward(inputPlaceholder)

epoch = 5000

error = tf.reduce_mean(0.5 * ( outputPlacehlder - predict ) ** 2)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train = optimizer.minimize(error)

inputTrain, inputTest, outputTrain, outputTest = train_test_split(inputData, target, test_size=0.1)
print(inputData.shape)
print(target.shape)
inputTrain, inputValidationTest, outputTrain, outputValidationTest = train_test_split(inputTrain, outputTrain, test_size=0.2)

print(inputTrain.shape)
print(inputValidationTest.shape)
print(inputTest.shape)


(1599, 4)
(1599, 5)
(1151, 4)
(288, 4)
(160, 4)


In [26]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #70% of the dataset - train
    for i in range(1, epoch + 1) :
        train_dict = {
            inputPlaceholder : inputTrain,
            outputPlacehlder : outputTrain
        }
        sess.run(train, feed_dict = train_dict)

        loss = sess.run(error, feed_dict = train_dict)

        if i % 100 == 0:
            print("Epoch : {}, loss : {}".format(i, loss))
        
        #20% of the dataset - valid
        if i % 500 == 0:
            
            validation_dict = {
                inputPlaceholder : inputValidationTest,
                outputPlacehlder : outputValidationTest
            }
            sess.run(train, feed_dict = validation_dict)

            Validationloss = sess.run(error, feed_dict = validation_dict)
            
#             print("Validation Epoch : {}, loss : {}".format(i, Validationloss))
            if i == 500:
                lowestValidationLoss = Validationloss
            
                f=open("lowestValidationLoss.txt", "w")
                f.write(str(lowestValidationLoss))
                f.close()
            
            if Validationloss < lowestValidationLoss:
                lowestValidationLoss = Validationloss
                f=open("lowestValidationLoss.txt", "w")
                f.write(str(lowestValidationLoss))
                f.close()
            
                
    #10% of the dataset - evaluation
    matches = tf.equal(tf.argmax(outputPlacehlder,axis = 1), tf.argmax(predict,axis = 1))
    accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))
    print(matches)
    feed_test = {
        inputPlaceholder: inputTest,
        outputPlacehlder: outputTest
    }

    print("accuracy: {}".format(sess.run(accuracy, feed_dict = feed_test) *100 ))
    


Epoch : 100, loss : 0.267198771238327
Epoch : 200, loss : 0.26713284850120544
Epoch : 300, loss : 0.2671055495738983
Epoch : 400, loss : 0.2670874297618866
Epoch : 500, loss : 0.26707392930984497
Epoch : 600, loss : 0.267062783241272
Epoch : 700, loss : 0.26705461740493774
Epoch : 800, loss : 0.2670477330684662
Epoch : 900, loss : 0.2670430839061737
Epoch : 1000, loss : 0.26703837513923645
Epoch : 1100, loss : 0.2670351564884186
Epoch : 1200, loss : 0.26703253388404846
Epoch : 1300, loss : 0.2670293152332306
Epoch : 1400, loss : 0.2670278251171112
Epoch : 1500, loss : 0.26702603697776794
Epoch : 1600, loss : 0.2670249044895172
Epoch : 1700, loss : 0.26702308654785156
Epoch : 1800, loss : 0.2670222520828247
Epoch : 1900, loss : 0.2670210003852844
Epoch : 2000, loss : 0.26702049374580383
Epoch : 2100, loss : 0.26701998710632324
Epoch : 2200, loss : 0.26701951026916504
Epoch : 2300, loss : 0.2670186161994934
Epoch : 2400, loss : 0.26701828837394714
Epoch : 2500, loss : 0.26701784133911133